# World Bank - Mortality Summarizing
Reduced '.csv' files from WB_data Notebook

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import requests
import matplotlib.pyplot as plt
from matplotlib import cbook
from scipy.stats import linregress
import scipy.stats as st
import numpy as np
import time

# Impor the OpenWeatherMap API key
from api_keys import geoapify_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

In [2]:
# WB data set
WB_africadata_2000_bins = Path("Resources\WB_africadata_2000_bins.csv")

In [3]:
# Read data file with the Pandas library
# encoding?, i.e encoding="ISO-8859-1"
WB_africadata_2000_df2 = pd.read_csv(WB_africadata_2000_bins)

In [4]:
WB_africadata_2000_df2

,series_id,country_code,country_name,year,value,Year Group
0,SH.HIV.INCD.TL,KEN,Kenya,2017,3.400000e+04,2016 - 2020
1,SH.HIV.INCD.TL,LBR,Liberia,2017,1.600000e+03,2016 - 2020
2,SH.HIV.INCD.TL,LSO,Lesotho,2017,1.000000e+04,2016 - 2020
3,SH.HIV.INCD.TL,MDG,Madagascar,2017,7.200000e+03,2016 - 2020
4,SH.HIV.INCD.TL,MOZ,Mozambique,2017,1.300000e+05,2016 - 2020
...,...,...,...,...,...,...
652157,VA.NO.SRC,WSM,Samoa,2006,3.000000e+00,2006 - 2010
652158,FM.LBL.BMNY.GD.ZS,TCD,Chad,2010,1.146678e+01,2006 - 2010
652159,NE.CON.PRVT.CN.AD,SDN,Sudan,2000,2.199464e+10,2000 - 2005
652160,NE.CON.PRVT.CN.AD,SDN,Sudan,2003,3.190503e+10,2000 - 2005


# Slicing of the Data
###In order to analyze the data and to understand the impact of the Foreign Aid with regards to poverty rate, literacy and mortality, the data was sliced by the "series_id". The deifinition of the diferent "series_id" values (indicators) is described in a second file downloaded from the Nasdaq API and saved as 'WB_metadata_df'.

###To facilitate/expedite the search and filtering of the relevant a function was created to search the text strings : "key_word". This function is case sensitive and it's used is combined with the review of the search output selection to identify the indicators that best match our work objective.

In [5]:
# WB id series definition data set
WB_metadata = Path("Resources/WB_METADATA_f7ce7fba293ccc6eb39cdf15fb097982.csv")

In [6]:
# Read data file with the Pandas library
WB_metadata_df = pd.read_csv(WB_metadata)

In [7]:
WB_metadata_df

,series_id,name,description
0,DC.DAC.DEUL.CD,"Net bilateral aid flows from DAC donors, Germa...",Net bilateral aid flows from DAC donors are th...
1,RQ.STD.ERR,Regulatory Quality: Standard Error,Regulatory Quality captures perceptions of the...
2,EG.USE.PCAP.KG.OE,Energy use (kg of oil equivalent per capita),Energy use refers to use of primary energy bef...
3,EN.POP.EL5M.UR.ZS,Urban population living in areas where elevati...,Urban population below 5m is the percentage of...
4,per_lm_alllm.cov_q1_tot,Coverage of unemployment benefits and ALMP in ...,Coverage of unemployment benefits and active l...
...,...,...,...
1479,SL.TLF.BASC.FE.ZS,"Labor force with basic education, female (% of...",The ratio of the labor force with basic educat...
1480,SL.GDP.PCAP.EM.KD,GDP per person employed (constant 2021 PPP $),GDP per person employed is gross domestic prod...
1481,SL.TLF.INTM.MA.ZS,"Labor force with intermediate education, male ...",The ratio of the labor force with intermediate...
1482,SL.UEM.NEET.FE.ZS,"Share of youth not in education, employment or...","Share of youth not in education, employment or..."


In [8]:
# This function will search within a text string for a given 'key word'. It is case sensitive

def key_word(df, search_col, word_txt, result_col):
    i=0   
    result_ls = []
    for item in df[search_col]:
        text_ls = [x.strip() for x in df[search_col][i].split()]
        if word_txt in text_ls:
            result_ls.append(df[result_col][i])
        i+=1
    if i >= df[search_col].count():
        if len(result_ls) > 0:
            return result_ls
        else:
            return print("Keyword Not found")

In [9]:
Mortality_search = key_word(WB_metadata_df, 'name', 'Mortality', 'series_id')

In [10]:
#len(Mortality_search)

In [11]:
Mortality_search2= WB_metadata_df[WB_metadata_df['series_id'].isin(Mortality_search)]

In [12]:
#Mortality_search2

# Selection of Indicators for Analysis

###Indicators are selected out of the search result and used to create redu ed data sets. In the case of the Mortality indicator, we selected three indicators : mortality under age 5-females only, mortality under age 5-males only, and mortality under age 5-combined. Values listed are the rate per 100,000 parts.

###In order to organize the data, we prepared 6 data sets. One for each indicator.

In [13]:
# Thought these indicators might provide more data for the Mortality analysis: rate is per 100000 females, males, population
#SH.DYN.MORT.FE - mortality under age 5, females
#SH.DYN.MORT.MA - mortality under age 5, males
#SH.DYN.MORT - mortality under age 5, combined

## Mortality Rate < 5 yr (per 100Th) - Females

In [14]:
WB_Mort_fem_df = WB_africadata_2000_df2.loc[WB_africadata_2000_df2['series_id'] == "SH.DYN.MORT.FE"]
#WB_Mort_fem_df = WB_Mort_fem_df.rename(columns = {'value': 'Mortality Rate < 5 yr (per 100Th)'})
WB_Mort_fem_df

,series_id,country_code,country_name,year,value,Year Group
127992,SH.DYN.MORT.FE,MDG,Madagascar,2016,60.1,2016 - 2020
196033,SH.DYN.MORT.FE,TUN,Tunisia,2018,14.1,2016 - 2020
301671,SH.DYN.MORT.FE,AGO,Angola,2000,190.9,2000 - 2005
301672,SH.DYN.MORT.FE,BFA,Burkina Faso,2000,171.5,2000 - 2005
301673,SH.DYN.MORT.FE,BWA,Botswana,2000,73.9,2000 - 2005
...,...,...,...,...,...,...
302571,SH.DYN.MORT.FE,UGA,Uganda,2022,36.1,2021 - 2023
302572,SH.DYN.MORT.FE,WSM,Samoa,2022,14.4,2021 - 2023
302573,SH.DYN.MORT.FE,ZMB,Zambia,2022,50.7,2021 - 2023
302574,SH.DYN.MORT.FE,ZWE,Zimbabwe,2022,43.3,2021 - 2023


In [15]:
reduc_Mort_fem_df = WB_Mort_fem_df.reset_index()
reduc_Mort_fem_df.drop(columns=['index','series_id'], inplace=True)
reduc_Mort_fem_df

,country_code,country_name,year,value,Year Group
0,MDG,Madagascar,2016,60.1,2016 - 2020
1,TUN,Tunisia,2018,14.1,2016 - 2020
2,AGO,Angola,2000,190.9,2000 - 2005
3,BFA,Burkina Faso,2000,171.5,2000 - 2005
4,BWA,Botswana,2000,73.9,2000 - 2005
...,...,...,...,...,...
708,UGA,Uganda,2022,36.1,2021 - 2023
709,WSM,Samoa,2022,14.4,2021 - 2023
710,ZMB,Zambia,2022,50.7,2021 - 2023
711,ZWE,Zimbabwe,2022,43.3,2021 - 2023


In [16]:
Mort_fem_table = reduc_Mort_fem_df.pivot(index='year', columns='country_name', values='value')
Mort_fem_table

country_name,Angola,Botswana,Burkina Faso,Central African Republic,Chad,Djibouti,"Egypt, Arab Rep.",Ethiopia,Gabon,Ghana,...,Senegal,Sierra Leone,Somalia,Sudan,Tanzania,Togo,Tunisia,Uganda,Zambia,Zimbabwe
year,,,,,,,,,,,,,,,,,,,,,
2000,190.9,73.9,171.5,157.5,175.0,92.9,44.8,129.8,77.6,92.0,...,122.7,214.7,164.3,95.9,122.9,111.0,26.5,134.3,147.0,87.8
2001,185.3,72.9,167.4,154.9,171.6,90.9,42.1,123.8,75.9,88.2,...,116.2,209.8,164.4,92.6,115.2,107.7,24.7,127.9,138.4,86.0
2002,178.7,71.1,162.8,152.1,168.1,88.6,39.7,117.7,74.1,84.4,...,108.5,204.5,164.5,89.3,107.3,104.5,23.2,120.8,128.1,84.7
2003,171.1,70.6,157.8,148.8,164.6,86.3,37.6,111.4,72.2,81.2,...,100.6,199.1,164.6,86.3,99.9,101.5,21.8,113.3,117.9,83.9
2004,162.8,69.7,152.3,145.1,161.3,83.9,35.7,105.2,70.1,78.3,...,92.9,193.5,164.6,83.2,93.2,98.5,20.7,105.6,107.9,84.1
2005,154.2,65.5,146.0,141.2,157.9,81.6,34.1,99.2,68.0,75.8,...,85.7,187.4,164.5,80.4,87.1,95.7,19.7,98.2,99.0,85.0
2006,145.1,61.2,139.6,136.7,154.6,79.2,32.5,93.3,66.0,73.5,...,79.4,181.0,163.9,77.7,81.7,93.1,18.9,91.3,91.5,87.1
2007,136.1,59.5,133.3,132.7,150.8,76.7,31.1,87.8,63.6,71.2,...,73.8,174.1,161.9,75.1,76.5,90.4,18.3,85.1,86.1,88.0
2008,127.3,59.0,127.1,128.8,147.1,74.4,29.6,82.8,61.3,68.6,...,68.9,167.0,158.6,72.7,72.0,87.8,17.8,79.3,81.5,87.0


In [17]:
# Save the DataFrame as a CSV
# Note: To avoid any issues later, use encoding="utf-8"
Mort_fem_table.to_csv("Resources\Mort_fem_table.csv", encoding="utf-8", index=False)

## Mortality Rate < 5 yr (per 100Th) - Males

In [18]:
WB_Mort_ma_df = WB_africadata_2000_df2.loc[WB_africadata_2000_df2['series_id'] == "SH.DYN.MORT.MA"]
WB_Mort_ma_df

,series_id,country_code,country_name,year,value,Year Group
463915,SH.DYN.MORT.MA,AGO,Angola,2000,214.2,2000 - 2005
463916,SH.DYN.MORT.MA,BFA,Burkina Faso,2000,184.1,2000 - 2005
463917,SH.DYN.MORT.MA,BWA,Botswana,2000,81.4,2000 - 2005
463918,SH.DYN.MORT.MA,CAF,Central African Republic,2000,172.5,2000 - 2005
463919,SH.DYN.MORT.MA,DJI,Djibouti,2000,106.0,2000 - 2005
...,...,...,...,...,...,...
464623,SH.DYN.MORT.MA,TZA,Tanzania,2022,44.2,2021 - 2023
464624,SH.DYN.MORT.MA,UGA,Uganda,2022,44.7,2021 - 2023
464625,SH.DYN.MORT.MA,WSM,Samoa,2022,17.7,2021 - 2023
464626,SH.DYN.MORT.MA,ZMB,Zambia,2022,60.2,2021 - 2023


In [19]:
reduc_Mort_ma_df = WB_Mort_ma_df.reset_index()
reduc_Mort_ma_df.drop(columns=['index','series_id'], inplace=True)
reduc_Mort_ma_df

,country_code,country_name,year,value,Year Group
0,AGO,Angola,2000,214.2,2000 - 2005
1,BFA,Burkina Faso,2000,184.1,2000 - 2005
2,BWA,Botswana,2000,81.4,2000 - 2005
3,CAF,Central African Republic,2000,172.5,2000 - 2005
4,DJI,Djibouti,2000,106.0,2000 - 2005
...,...,...,...,...,...
708,TZA,Tanzania,2022,44.2,2021 - 2023
709,UGA,Uganda,2022,44.7,2021 - 2023
710,WSM,Samoa,2022,17.7,2021 - 2023
711,ZMB,Zambia,2022,60.2,2021 - 2023


In [20]:
Mort_ma_table = reduc_Mort_ma_df.pivot(index='year', columns='country_name', values='value')
Mort_ma_table

country_name,Angola,Botswana,Burkina Faso,Central African Republic,Chad,Djibouti,"Egypt, Arab Rep.",Ethiopia,Gabon,Ghana,...,Senegal,Sierra Leone,Somalia,Sudan,Tanzania,Togo,Tunisia,Uganda,Zambia,Zimbabwe
year,,,,,,,,,,,,,,,,,,,,,
2000,214.2,81.4,184.1,172.5,192.8,106.0,48.5,152.0,90.9,107.6,...,136.0,235.2,178.9,110.6,134.7,128.1,32.1,156.7,165.2,100.7
2001,207.8,79.9,180.1,169.7,189.3,103.7,45.9,145.6,89.0,103.5,...,129.1,229.3,179.0,107.0,126.5,124.4,29.9,149.4,156.3,98.7
2002,200.7,78.4,175.8,166.5,185.9,101.3,43.4,139.0,86.9,99.6,...,121.2,223.4,178.9,103.4,118.6,120.6,28.0,141.2,145.7,97.3
2003,192.7,78.0,170.6,163.1,182.6,98.9,41.2,132.5,84.8,96.2,...,112.9,217.2,178.8,99.8,111.0,117.0,26.3,132.6,134.9,96.3
2004,183.9,77.3,164.5,159.5,179.0,96.6,39.2,125.8,82.5,93.4,...,104.7,210.7,178.9,96.5,104.0,113.4,24.9,123.9,124.2,96.6
2005,174.6,73.1,158.0,155.8,175.3,94.0,37.4,119.2,80.1,90.7,...,97.1,204.1,178.8,93.3,97.6,110.1,23.6,115.5,114.5,97.5
2006,164.9,69.2,151.0,151.5,171.4,91.5,35.8,112.9,77.8,88.3,...,90.5,197.3,178.2,90.3,92.0,106.8,22.6,107.7,106.1,99.8
2007,155.3,68.1,144.1,147.4,167.6,89.0,34.3,106.8,75.2,85.7,...,84.7,190.3,176.2,87.5,86.6,103.8,21.8,100.6,100.2,100.6
2008,145.9,67.9,137.5,143.6,163.7,86.5,32.9,101.1,72.7,82.9,...,79.6,183.0,172.7,85.0,81.9,100.7,21.2,94.1,94.8,99.4


In [21]:
# Save the DataFrame as a CSV
# Note: To avoid any issues later, use encoding="utf-8"
Mort_ma_table.to_csv("Resources\Mort_ma_table.csv", encoding="utf-8", index=False)

In [22]:
#WB_Mort_ma_df = WB_Mort_fem_df.rename(columns = {'value': 'Mortality Rate < 5 yr (per 100Th)'})
#WB_Mort_ma_df

## Mortality Rate < 5 yr (per 100Th) - Combined

In [23]:
WB_Mort_combo_df = WB_africadata_2000_df2.loc[WB_africadata_2000_df2['series_id'] == "SH.DYN.MORT"]
WB_Mort_combo_df

,series_id,country_code,country_name,year,value,Year Group
526439,SH.DYN.MORT,AGO,Angola,2000,202.9,2000 - 2005
526440,SH.DYN.MORT,BFA,Burkina Faso,2000,178.0,2000 - 2005
526441,SH.DYN.MORT,BWA,Botswana,2000,77.8,2000 - 2005
526442,SH.DYN.MORT,CAF,Central African Republic,2000,165.2,2000 - 2005
526443,SH.DYN.MORT,DJI,Djibouti,2000,99.7,2000 - 2005
...,...,...,...,...,...,...
527149,SH.DYN.MORT,TZA,Tanzania,2022,40.5,2021 - 2023
527150,SH.DYN.MORT,UGA,Uganda,2022,40.5,2021 - 2023
527151,SH.DYN.MORT,WSM,Samoa,2022,16.1,2021 - 2023
527152,SH.DYN.MORT,ZMB,Zambia,2022,55.6,2021 - 2023


In [24]:
reduc_Mort_combo_df = WB_Mort_combo_df.reset_index()
reduc_Mort_combo_df.drop(columns=['index','series_id'], inplace=True)
reduc_Mort_combo_df

,country_code,country_name,year,value,Year Group
0,AGO,Angola,2000,202.9,2000 - 2005
1,BFA,Burkina Faso,2000,178.0,2000 - 2005
2,BWA,Botswana,2000,77.8,2000 - 2005
3,CAF,Central African Republic,2000,165.2,2000 - 2005
4,DJI,Djibouti,2000,99.7,2000 - 2005
...,...,...,...,...,...
708,TZA,Tanzania,2022,40.5,2021 - 2023
709,UGA,Uganda,2022,40.5,2021 - 2023
710,WSM,Samoa,2022,16.1,2021 - 2023
711,ZMB,Zambia,2022,55.6,2021 - 2023


In [25]:
Mort_combo_table = reduc_Mort_combo_df.pivot(index='year', columns='country_name', values='value')
Mort_combo_table

country_name,Angola,Botswana,Burkina Faso,Central African Republic,Chad,Djibouti,"Egypt, Arab Rep.",Ethiopia,Gabon,Ghana,...,Senegal,Sierra Leone,Somalia,Sudan,Tanzania,Togo,Tunisia,Uganda,Zambia,Zimbabwe
year,,,,,,,,,,,,,,,,,,,,,
2000,202.9,77.8,178.0,165.2,184.1,99.7,46.7,141.2,84.4,100.0,...,129.6,225.3,171.9,103.4,129.0,119.8,29.4,145.8,156.3,94.5
2001,197.0,76.5,173.9,162.5,180.7,97.4,44.0,135.0,82.6,96.0,...,122.8,219.9,171.9,100.0,121.0,116.2,27.4,139.0,147.6,92.5
2002,190.0,74.8,169.4,159.5,177.3,95.1,41.6,128.6,80.7,92.2,...,115.0,214.2,171.9,96.5,113.1,112.8,25.7,131.3,137.1,91.2
2003,182.3,74.4,164.3,156.1,173.9,92.8,39.4,122.2,78.6,88.9,...,106.9,208.4,171.9,93.2,105.6,109.4,24.1,123.2,126.6,90.3
2004,173.7,73.6,158.5,152.5,170.4,90.4,37.5,115.8,76.4,86.0,...,99.0,202.3,171.9,90.0,98.8,106.2,22.8,115.0,116.3,90.5
2005,164.7,69.4,152.2,148.7,166.9,88.0,35.8,109.5,74.2,83.4,...,91.6,196.0,171.8,87.0,92.5,103.1,21.7,107.1,107.0,91.4
2006,155.2,65.3,145.5,144.3,163.2,85.6,34.2,103.3,72.1,81.1,...,85.1,189.3,171.2,84.2,87.0,100.1,20.8,99.7,99.0,93.6
2007,145.9,64.0,138.8,140.2,159.5,83.1,32.7,97.6,69.6,78.7,...,79.4,182.4,169.3,81.5,81.6,97.2,20.1,93.0,93.3,94.5
2008,136.7,63.5,132.4,136.4,155.6,80.6,31.3,92.2,67.1,75.9,...,74.4,175.2,165.9,79.0,77.1,94.4,19.6,86.9,88.3,93.3


In [26]:
# Save the DataFrame as a CSV
# Note: To avoid any issues later, use encoding="utf-8"
Mort_combo_table.to_csv("Resources\Mort_combo_table.csv", encoding="utf-8", index=False)

In [27]:
#WB_Mort_combo_df = WB_Mort_fem_df.rename(columns = {'value': 'Mortality Rate < 5 yr (per 100Th)'})
#WB_Mort_combo_df

In [28]:
data4 = {
    '2000 Female Mortality Rate < 5 yr (per 100Th)': (Mort_fem_table.iloc[0, 0:31]),
    '2022 Female Mortality Rate < 5 yr (per 100Th)': (Mort_fem_table.iloc[-1, 0:31]),
    '2000 Male Mortality Rate < 5 yr (per 100Th)': (Mort_ma_table.iloc[0, 0:31]),
    '2022 Male Mortality Rate < 5 yr (per 100Th)': (Mort_ma_table.iloc[-1, 0:31]),
    '2000 Combined Mortality Rate < 5 yr (per 100Th)': (Mort_combo_table.iloc[0, 0:31]),
    '2022 Combined Mortality Rate < 5 yr (per 100Th)': (Mort_combo_table.iloc[-1, 0:31])
}
summary_table4 = pd.DataFrame(data4)
summary_table4

,2000 Female Mortality Rate < 5 yr (per 100Th),2022 Female Mortality Rate < 5 yr (per 100Th),2000 Male Mortality Rate < 5 yr (per 100Th),2022 Male Mortality Rate < 5 yr (per 100Th),2000 Combined Mortality Rate < 5 yr (per 100Th),2022 Combined Mortality Rate < 5 yr (per 100Th)
country_name,,,,,,
Angola,190.9,61.0,214.2,72.4,202.9,66.9
Botswana,73.9,35.1,81.4,42.3,77.8,38.7
Burkina Faso,171.5,73.9,184.1,83.3,178.0,78.8
Central African Republic,157.5,89.9,172.5,103.4,165.2,96.8
Chad,175.0,96.2,192.8,109.3,184.1,102.9
Djibouti,92.9,47.0,106.0,56.6,99.7,51.9
"Egypt, Arab Rep.",44.8,16.8,48.5,19.3,46.7,18.1
Ethiopia,129.8,40.5,152.0,51.7,141.2,46.2
Gabon,77.6,34.7,90.9,42.8,84.4,38.8


In [29]:
# Save the DataFrame as a CSV
# Note: To avoid any issues later, use encoding="utf-8"
summary_table4.to_csv("Resources\WB_mortality_summary.csv", encoding="utf-8", index=False)